In [1]:
from ss2hcsp.hcsp.simulator import SimInfo, exec_parallel
from ss2hcsp.hcsp.hcsp import HCSPOutput
from ss2hcsp.hcsp.graph_plot import graph
import random

In [2]:
def run_processes(infos, num_io_events, *, output):
    list_infos = []
    for name, ps in infos.items():
        list_infos.append(SimInfo(name, ps, outputs=[HCSPOutput(var) for var in output]))
    res = exec_parallel(list_infos, num_io_events=num_io_events)
    events = [event['str'] for event in res['trace'] if event['str'] not in ('start', 'step')]
    return res, events

We begin with a simple example of random walk on a 6-by-6 grid.

In [6]:
random.seed(1)
res, events = run_processes({
    'P0': """
    x := 0; y := 0; {
        z := bernoulli(0.5);
        if (z == 1 && x < 5) {
            x := x + 1;
        } else if (x > 0) {
            x := x - 1;
        }
        z := bernoulli(0.5);
        if (z == 1 && y < 5) {
            y := y + 1;
        } else if (y > 0) {
            y := y - 1;
        }
    }*(x != 5 || y != 5)"""
}, 100, output=["x", "y", "z"])

print("Number of steps:", len(res['time_series']['P0'][2:]))

Number of steps: 94


In [21]:
from tkinter import * 
from tkinter import ttk
import time

tk = Tk()
tk.title("Random walk")

canvas = Canvas(tk, width=360, height=360)
canvas.pack()

# Create lines
for i in range(6):
    canvas.create_line(0, 60*(i+1), 360, 60*(i+1))
    canvas.create_line(60*(i+1), 0, 60*(i+1), 360)

# Create regions
regions = dict()
for i in range(6):
    for j in range(6):
        regions[(i,j)] = canvas.create_rectangle(60*i, 60*j, 60*(i+1), 60*(j+1), fill="green")

# Obtain data
x, y = [], []
for item in res['time_series']['P0'][2:]:
    x.append(item['state']['x'])
    y.append(item['state']['y'])

# Create scale
prev_x, prev_y = 0, 0
scale_var = IntVar()
def set_scale(*args):
    global prev_x, prev_y
    step = scale_var.get()
    step_label.set("Step: %s" % step)
    canvas.itemconfigure(regions[(prev_x, prev_y)], fill="green")
    canvas.itemconfigure(regions[(x[step],y[step])], fill="blue")
    prev_x, prev_y = x[step], y[step]

s = ttk.Scale(tk, orient=HORIZONTAL, length=200, from_=0, to=len(x)-1, variable=scale_var, command=set_scale)
s.pack()
step_label = StringVar()
lb = ttk.Label(tk, textvariable=step_label)
lb.pack()
set_scale()

# Create run button
interrupt = True
def step(count):
    if interrupt or scale_var.get() == len(x)-1:
        return
    else:
        scale_var.set(scale_var.get() + 1)
        set_scale()
        tk.after(100, lambda: step(count + 1))

def stop_simulation():
    global interrupt
    interrupt = True
    
def run_simulation(*args):
    global interrupt
    interrupt = False
    step(0)

run_button = ttk.Button(tk, text="Play", command=run_simulation)
run_button.pack()

stop_button = ttk.Button(tk, text="Stop", command=stop_simulation)
stop_button.pack()

tk.mainloop()

Next, we consider a slightly more complicated example, with an opponent that need to be avoided.

Below, variable $x$ and $y$ are position of the robot, and variable $rx$ and $ry$ are position of the opponent.

Change seed to 2 to see an example where the robot is caught by the opponent.

In [39]:
random.seed(1)
res, events = run_processes({
    'P0': """
    x := 0; y := 0;
    rx := 3; ry := 0; dir := 1; {
        z := bernoulli(0.5);
        if (z == 1 && x < 5) {
            x := x + 1;
        } else if (x > 0) {
            x := x - 1;
        }
        z := bernoulli(0.5);
        if (z == 1 && y < 5) {
            y := y + 1;
        } else if (y > 0) {
            y := y - 1;
        }
        ry := ry + dir;
        if (ry == 5) {
            dir := -1;
        } else if (ry == 0) {
            dir := 1;
        }
    }*(!(x == 5 && y == 5 || x == rx && y == ry))"""
}, 100, output=["x", "y", "rx", "ry"])

print("Number of steps:", len(res['time_series']['P0'][5:]))

Number of steps: 129


In [40]:
from tkinter import * 
from tkinter import ttk
import time

tk = Tk()
tk.title("Random walk")

canvas = Canvas(tk, width=360, height=360)
canvas.pack()

# Create lines
for i in range(6):
    canvas.create_line(0, 60*(i+1), 360, 60*(i+1))
    canvas.create_line(60*(i+1), 0, 60*(i+1), 360)

# Create regions
regions = dict()
for i in range(6):
    for j in range(6):
        regions[(i,j)] = canvas.create_rectangle(60*i, 60*j, 60*(i+1), 60*(j+1), fill="green")

# Obtain data
x, y, rx, ry = [], [], [], []
for item in res['time_series']['P0'][4:]:
    x.append(item['state']['x'])
    y.append(item['state']['y'])
    rx.append(item['state']['rx'])
    ry.append(item['state']['ry'])

# Create scale
prev_x, prev_y, prev_rx, prev_ry = 0, 0, 3, 0
scale_var = IntVar()
def set_scale(*args):
    global prev_x, prev_y, prev_rx, prev_ry
    step = scale_var.get()
    step_label.set("Step: %s" % step)
    canvas.itemconfigure(regions[(prev_x, prev_y)], fill="green")
    canvas.itemconfigure(regions[(prev_rx, prev_ry)], fill="green")
    canvas.itemconfigure(regions[(x[step],y[step])], fill="blue")
    canvas.itemconfigure(regions[(rx[step],ry[step])], fill="red")
    prev_x, prev_y = x[step], y[step]
    prev_rx, prev_ry = rx[step], ry[step]

s = ttk.Scale(tk, orient=HORIZONTAL, length=200, from_=0, to=len(x)-1, variable=scale_var, command=set_scale)
s.pack()
step_label = StringVar()
lb = ttk.Label(tk, textvariable=step_label)
lb.pack()
set_scale()

# Create run button
interrupt = True
def step(count):
    if interrupt or scale_var.get() == len(x)-1:
        return
    else:
        scale_var.set(scale_var.get() + 1)
        set_scale()
        tk.after(100, lambda: step(count + 1))

def stop_simulation():
    global interrupt
    interrupt = True
    
def run_simulation(*args):
    global interrupt
    interrupt = False
    step(0)

run_button = ttk.Button(tk, text="Play", command=run_simulation)
run_button.pack()

stop_button = ttk.Button(tk, text="Stop", command=stop_simulation)
stop_button.pack()

tk.mainloop()